# Description

Visualize the Eyring-Kramers formula and compare it with first passage times obtained by using, for instance, `distfpt.jl` from the `scripts` folder.

In [ ]:
using Plots
using Printf
using Random
using LaTeXStrings
using DSP
using LinearAlgebra
using ForwardDiff
using Statistics
using JLD2
using StatsPlots

In [ ]:
default(xtickfontsize=14, ytickfontsize=14,
    guidefontsize=14,
    legendfontsize=12, lw=2, ms=8)

In [ ]:
push!(LOAD_PATH, "../GeneralizedKuramoto");
push!(LOAD_PATH, "../QProcesses");
push!(LOAD_PATH, "../GraphMatrices");
push!(LOAD_PATH, "../TwistedStates");
push!(LOAD_PATH, "../ClassicalKuramoto");

In [ ]:
using GeneralizedKuramoto
using QProcesses
using GraphMatrices
using TwistedStates
using ClassicalKuramoto

# Set parameters

In [ ]:
n = 10; # number of sites
k = 1;  # interaction range, k=1 is nearest neighbors
q = 1;  # initial twisted state
α = 0;  # attractive coupling

# Precompute constants
To use the Eyring-Kramers formula, we must compute the energy barrier, `ΔE`  and the prefactor, `C`, in 
$$
MFPT \asymp C \exp\left(\Delta E\right)
$$

In [ ]:

q1 = TwistedStates.construct_q_twisted(n, q);
q0 = TwistedStates.construct_q_twisted(n, q - 1);

l = q - 1;
q_saddle = (2 * l + 1) * n / (2 * (n - 2)) * (0:n-1) / n; #analytic
# prepare interaction matrix
K = GraphMatrices.discrete_knn_spmatrix(n, k);

# compute hessian by automatic differentiation
E = u -> ClassicalKuramoto.energy(u, K, α);
HessE(u) = ForwardDiff.hessian(E, u);
gradE(u) = ForwardDiff.gradient(E, u);

@show E(q1);
@show E(q0);
@show E(q_saddle);
@show E(q1) - E(q0);
@show E(q_saddle) - E(q1);
@show E(q_saddle) - E(q0);
ΔE = E(q_saddle) - E(q1);
@show ΔE;

evals_min = eigvals(HessE(q1));
nonzero_evals_min = evals_min[2:end];
evals_saddle = eigvals(HessE(q_saddle));
nonzero_evals_saddle = [evals_saddle[1]; evals_saddle[3:end]];
C = (2 * π / n) / abs(nonzero_evals_saddle[1]) * sqrt(abs(prod(nonzero_evals_saddle)) / prod(nonzero_evals_min));
@show C;

# Load data

In [ ]:
filepath = "/path/to/data";

β_vals = [10, 20, 30, 40, 50];
n_samples = 10^4;

filenames = ["fpt2_n10_q1_k1_N10000_beta10_dt0_01_tmax10000_s1000.jld2",
    "fpt2_n10_q1_k1_N10000_beta20_dt0_01_tmax10000_s1000.jld2",
    "fpt2_n10_q1_k1_N10000_beta30_dt0_01_tmax10000_s1000.jld2",
    "fpt2_n10_q1_k1_N10000_beta40_dt0_01_tmax10000_s1000.jld2",
    "fpt2_n10_q1_k1_N10000_beta50_dt0_01_tmax10000_s1000.jld2"]
t_exit_vals = Array{Float64,1}[];
for name in filenames
    data = jldopen(filepath * "/" * name)
    @show var(data["E_exit_values"])
    @show mean(data["t_exit_values"])
    @show maximum(data["t_exit_values"])
    push!(t_exit_vals, data["t_exit_values"])
end

T_vals = zeros(n_samples, length(filenames),);

for (i, name) in enumerate(filenames)
    data = jldopen(filepath * "/" * name)
    T_vals[:, i] .= data["t_exit_values"]
end

# Visualize

In [ ]:
bar_width=5;

boxplot([β_vals[1]], T_vals[:, 1], legend=:topleft, yscale=:log10, bar_width=bar_width, label="Samples")
for j in 2:length(β_vals)
    boxplot!([β_vals[j]], T_vals[:, j], color=1, bar_width=bar_width, label="")
end
β_ = LinRange(5, 55, 10)
ylims!(10^-1.5, 10^2.5)
plot!(β_, C * exp.(β_ * ΔE),
    label=L"C(q,n)\exp\left(H_{q+1}/\epsilon\right)", color=2, lw=4, ls=:dash)
xlabel!(L"1/\epsilon")
yticks!(10.0 .^ (-1:1:2))
# ylabel!("MFPT")
ylabel!("First Passage Time")
